In [ ]:
!pip install yfinance --quiet
!pip install pmdarima --quiet

In [ ]:
!pip install statsmodels==0.11.0rc1 --quiet
!pip install -Iv pulp==1.6.8 --quiet

In [ ]:
import yfinance as yf

# getting data from Yahoo Finance
stock_name = 'IBM' 
data = yf.download(stock_name, start="2000-01-01", end="2023-05-10")

In [ ]:
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pmdarima as pm
plt.style.use('fivethirtyeight')
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import ADFTest
from pmdarima.datasets import load_wineind
import random

# Модель ARIMA

In [ ]:
data_adf = data.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1)
data_adf = data_adf['Close']

from pmdarima.arima import ADFTest
adf_test = ADFTest(alpha = 0.05)
adf_test.should_diff(data_adf)

In [ ]:
def arima(stock_name, data):
    df_close = data['Close']
    
    # Split data into train and test set (90% - train, 10% - test)
    df_log = df_close
    train_data, test_data = df_log[3:int(len(df_log) * 0.9)], df_log[int(len(df_log) * 0.9 - 1):]
    test_values = len(df_log) * 0.01 + 1.0
    x_train = list(range(0, int(0.9*len(data))))
    x_test = list(range(int(0.9*len(data) - 1), int(len(data))))
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x_train, y=train_data, mode='lines+markers', marker=dict(size=4),  name='train', marker_color='#39304A'))
    fig.add_trace(go.Scatter(x=x_test, y=test_data, mode='lines+markers', marker=dict(size=4), name='test', marker_color='#A98D75'))
    fig.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  plot_bgcolor='#FFFFFF',  
                  xaxis=dict(gridcolor = 'lightgrey'),
                  yaxis=dict(gridcolor = 'lightgrey'),
                  title_text = f'{stock_name} ARIMA data', title_x = 0.5,
                  xaxis_title="Timestep",
                  yaxis_title="Stock price",
                  margin=dict(l=0, r=0, t=30, b=0))
    fig.show()
    
    model =  pm.auto_arima(df_log,start_p=0, d=None, start_q=0, 
                          max_p=2, max_d=2, max_q=2, start_P=0, 
                          D=1, start_Q=0, max_P=2, max_D=2,
                          max_Q=2, m=7, seasonal=True, 
                          error_action='warn',trace = True,
                          supress_warnings=True,stepwise = True,
                          random_state=20,n_fits = 50 )

    model.summary()

    exo_data = data['Volume']
    exo_data = exo_data[int(len(exo_data) * 0.9):]
    
    preds = model.predict(n_periods = 22, X = exo_data)

    preds = np.vstack(preds)
    hist_data = yf.download(stock_name, start="2000-01-01", end="2023-05-10")
    hist_data = hist_data.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1)
    hist_data = hist_data['Close']
    hist_data = np.array(hist_data)
    
    rmse = np.sqrt(np.mean(((preds - hist_data) ** 2)))
    print(f'RMSE ARIMA: {rmse}')
    
    # build graphs
    preds_gr = np.reshape(preds, (22,))
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=list(range(0, 21)), y=hist_data, mode='lines+markers',  name='historical', marker_color='#39304A'))
    fig.add_trace(go.Scatter(x=list(range(0, 21)), y=preds_gr, mode='lines+markers', name='predictions', marker_color='#FFAA00'))
    fig.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  plot_bgcolor='#FFFFFF',  
                  xaxis=dict(gridcolor = 'lightgrey'),
                  yaxis=dict(gridcolor = 'lightgrey'),
                  title_text = f'{stock_name} ARIMA prediction', title_x = 0.5,
                  xaxis_title="Timestep",
                  yaxis_title="Stock price",
                  margin=dict(l=0, r=0, t=30, b=0))
    fig.show()

    return preds, rmse

In [ ]:
arima_pred, arima_rmse = arima(stock_name, data)
print(arima_pred.shape)

#SARIMAX

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
data3 = data['Close']
    
# Split data into train and test set (90% - train, 10% - test)
train3_data, test3_data = data3[3:int(len(data3) * 0.9)], data3[int(len(data3) * 0.9):]
#test_values = len(data3) * 0.01 + 1.0
x_train = list(range(0, int(0.9*len(data))))
x_test = list(range(int(0.9*len(data)), int(len(data3))))

exo_data = data['Volume']
exo_data = exo_data[int(len(exo_data) * 0.9):]

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_train, y=train3_data, mode='lines+markers', marker=dict(size=4),  name='train', marker_color='#39304A'))
fig.add_trace(go.Scatter(x=x_test, y=test3_data, mode='lines+markers', marker=dict(size=4), name='test', marker_color='#A98D75'))
fig.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  plot_bgcolor='#FFFFFF',  
                  xaxis=dict(gridcolor = 'lightgrey'),
                  yaxis=dict(gridcolor = 'lightgrey'),
                  title_text = f'{stock_name} SARIMAX data', title_x = 0.5,
                  xaxis_title="Timestep",
                  yaxis_title="Stock price",
                  margin=dict(l=0, r=0, t=30, b=0))
fig.show()
    
model = SARIMAX(train3_data, order=(3, 1, 2))

arima_model = model.fit(X = exo_data, disp=-1)

print(arima_model.summary())


preds3 = arima_model.predict(n_periods=22, alpha=0.05)

preds3 = np.vstack(preds3)
preds3 = preds3[-22:]
hist_data = yf.download(stock_name, start="1970-01-01", end="2023-05-10")
hist_data = hist_data.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1)
hist_data = hist_data['Close']
hist_data = np.array(hist_data)
    
rmse = np.sqrt(np.mean(((preds3 - hist_data) ** 2)))
print(f'RMSE SARIMAX: {rmse}')
    
preds_gr = np.reshape(preds3, (22,))
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(0, 21)), y=hist_data, mode='lines+markers', name='historical', marker_color='#39304A'))
fig.add_trace(go.Scatter(x=list(range(0, 21)), y=preds_gr, mode='lines+markers', name='predictions', marker_color='#FFAA00'))
fig.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  plot_bgcolor='#FFFFFF',  
                  xaxis=dict(gridcolor = 'lightgrey'),
                  yaxis=dict(gridcolor = 'lightgrey'),
                  title_text = f'{stock_name} SARIMAX prediction', title_x = 0.5,
                  xaxis_title="Timestep",
                  yaxis_title="Stock price",
                  margin=dict(l=0, r=0, t=30, b=0))
fig.show()

# Модель экспоненциального сглаживания

In [ ]:
%matplotlib inline

import math
import matplotlib
import multiprocessing
import numpy as np
import pandas as pd
import pickle
import time

from collections import defaultdict
from datetime import date, datetime, timedelta
from joblib import Parallel, delayed
from matplotlib import pyplot as plt
from pylab import rcParams

#### Input params ##################
H = 21
train_size = 252*3              # Use 3 years of data as train set. Note there are about 252 trading days in a year
val_size = 252                  # Use 1 year of data as validation set

# alpha - smoothing coeff
alphaMax = 0.999
alphaMin = 0.01
alphaStep = 0.01

# beta - trend coeff
betaMax = 0.999
betaMin = 0.01
betaStep = 0.01

# gamma - seasonality coeff
gammaMax = 0.99
gammaMin = 0.1
gammaStep = 0.1   

L = 252                        # seasonality period

# for plot display
daysBackward = 30
daysForward = 60

i_list = range(1008, 1008+84*5+42+1, 42) # we want to do a forecast on these days

fontsize = 14
ticklabelsize = 14
####################################

train_val_size = train_size + val_size # Size of train+validation set

In [ ]:
import yfinance as yf

# getting data from Yahoo Finance
stock_name = 'IBM' 
df = yf.download(stock_name, start="2000-01-01", end="2023-05-10").reset_index()

In [ ]:
def get_mape(y_true, y_pred): 
    """
    Compute mean absolute percentage error (MAPE)
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def get_mae(a, b):
    """
    Comp mean absolute error e_t = E[|a_t - b_t|]. a and b can be lists.
    Returns a vector of len = len(a) = len(b)
    """
    return np.mean(abs(np.array(a)-np.array(b)))

def get_rmse(a, b):
    """
    Comp RMSE. a and b can be lists.
    Returns a scalar.
    """
    return math.sqrt(np.mean((np.array(a)-np.array(b))**2))

def initial_trend(series, L):
    """
    Initial trend, b_1 = ( (y_L+1 - y_1)/L + (y_L+2 - y_2)/L + ... + (y_L+L - y_L)/L ) / L
    """
    sum = 0.0
    for i in range(L):
        sum += float(series[i+L] - series[i]) / L
        
    return sum / L

def initial_seasonal_components(series, L):
    """
    Initial seasonal index, 
    I_t = ( y_t-A_1 + y_{t+L}-A_2 + ... + y_{t+(P-1)L}-A_P ) / P, 
    t = 1, 2, ..., L
    Here P is the number of seasons we have in the series.
    For example, for sales data, we have 2018 Q1, 2018 Q2, 2018 Q3, 2018 Q4 data. These 4 points represent 1 season.
    A_1 is the mean of the values in the first season, and so on.
    Returns the seasonal components of length L in a list
    """
    seasonals = []
    season_averages = []
    n_seasons = int(len(series)/L)
    
    # compute season averages
    for j in range(n_seasons):
        season_averages.append(sum(series[L*j:L*j+L])/float(L))
    
    # compute initial values
    for i in range(L):
        sum_of_vals_over_avg = 0.0
        for j in range(n_seasons):
            sum_of_vals_over_avg += series[L*j+i]-season_averages[j]
        seasonals.append(sum_of_vals_over_avg/n_seasons)
        
    return seasonals

def triple_exponential_smoothing(series, L, H, alpha=0.3, beta=0.3, gamma=0.3, return_all=False):
    """
    Overall smoothing:  S_t = alpha*(y_t - I_{t-L}) + (1-alpha)(S_{t-1} + b_{t-1}})
    Trend smoothing:    b_t = beta*(S_t - S_{t-1}) + (1-beta)*b_{t-1}
    Seasonal smoothing: I_t = gamma*(y_t - S_t) + (1-gamma)*I_{t-L}
    Forecast:           F_{t+m} = S_t + m*b_t + I_{t-L+m}, m >= 1
    Note here m has to be < len(series)
    result[len(series)] is the estimate of series[len(series)]
    The length of result is len(series) + H, where H >= 1
    """
    result = [0, series[0]]
    smooth = series[0]
    trend = initial_trend(series, L)
    seasonals = initial_seasonal_components(series, L)
    seasonals.append(seasonals[0]) # To make the seasonals elements align with series elements
    for n in range(1, len(series)+H-1):
        if n >= len(series): # we are forecasting
            m = n - len(series) + 2
            result.append(smooth + m*trend + seasonals[n+1])
        else:
            val = series[n]
            last_smooth, smooth = smooth, alpha*(val-seasonals[n]) + (1-alpha)*(smooth+trend)
            trend = beta * (smooth-last_smooth) + (1-beta)*trend
            seasonals.append(gamma*(val-smooth) + (1-gamma)*seasonals[n])
            result.append(smooth + trend + seasonals[n+1])
            # e.g. result[2] uses series[1], seasonals[1], and seasonals[2]
            # ie. result[2] is the estimate of series[2]
            # e.g. result[len(series)] uses series[len(series)-1], seasonals[len(series)-1], and seasonals[len(series)] 
            # ie. result[len(series)] is the estimate of series[len(series)]
            
    if return_all == True:
        return result, seasonals
    else:
        return result[len(series):len(series)+H], seasonals

def get_error_metrics(series, train_size, L, H, alpha, beta, gamma):
    """
    Given a series consisting of both train+validation, do predictions of forecast horizon H on the validation set, 
    at H/2 intervals.
    Inputs
        series     : series to forecast, with length = (train_size + val_size)
        train_size : length of series to use as train ie. train set is series[:train_size]
        L          : period
        H          : forecast horizon
        alpha      : smoothing coeff
        beta       : trend coeff
        gamma      : seasonality coeff
    Outputs
        mean of rmse, mean of mape, mean of mae
    """
    # Predict using single exponential smoothing, and compute error metrics also
    rmse = [] # root mean square error
    mape = [] # mean absolute percentage error
    mae = []  # mean absolute error
    preds_dict = {}
    
    for i in range(train_size, len(series)-H+1, int(H/2)):
        preds_list, seasonals = triple_exponential_smoothing(series[i-train_size:i], L, H, alpha, beta, gamma)
        
        rmse.append(get_rmse(series[i:i+H], preds_list))
        mape.append(get_mape(series[i:i+H], preds_list))
        mae.append(get_mae(series[i:i+H], preds_list))
        preds_dict[i] = preds_list
    
    return np.mean(rmse), np.mean(mape), np.mean(mae), preds_dict    
    
def hyperparam_tune_alpha_beta_gamma(series, train_size, L, H):
    """
    Given a series, tune hyperparameter alpha, fit and predict
    Inputs
        series     : series to forecast, with length = (train_size + val_size)
        train_size : length of series to use as train ie. train set is series[:train_size]
        L          : period
        H          : forecast horizon
    Outputs
        optimum hyperparameters, error metrics dataframe
    """
    err_dict = defaultdict(list)
    alpha = alphaMin
    while alpha <= alphaMax:
        beta = betaMin
        while beta <= betaMax:
            gamma = gammaMin
            while gamma <= gammaMax:
                rmse_mean, mape_mean, mae_mean, _ = get_error_metrics(series, train_size, L, H, alpha, beta, gamma)
        
                # Append alpha and beta
                err_dict['alpha'].append(alpha)
                err_dict['beta'].append(beta)
                err_dict['gamma'].append(gamma)
    
                # Compute error metrics
                err_dict['rmse'].append(rmse_mean)
                err_dict['mape'].append(mape_mean)
                err_dict['mae'].append(mae_mean)
                
                # Increase gamma by one step
                gamma = gamma + gammaStep
            
            # Increase beta by one step
            beta = beta + betaStep
        
        # Increase alpha by one step
        alpha = alpha + alphaStep
    
    # Convert to dataframe
    err_df = pd.DataFrame(err_dict)
    
    # Get min RMSE
    rmse_min = err_df['rmse'].min()
    
    return err_df[err_df['rmse'] == rmse_min]['alpha'].values[0], \
           err_df[err_df['rmse'] == rmse_min]['beta'].values[0], \
           err_df[err_df['rmse'] == rmse_min]['gamma'].values[0], \
           err_df

In [ ]:
def get_error_metrics2(series, train_size, L, H, alpha, beta, gamma):
    """
    Given a series consisting of both train+validation, do predictions of forecast horizon H on the validation set, 
    at H/2 intervals.
    Inputs
        series     : series to forecast, with length = (train_size + val_size)
        train_size : length of series to use as train ie. train set is series[:train_size]
        L          : period
        H          : forecast horizon
        alpha      : smoothing coeff
        beta       : trend coeff
        gamma      : seasonality coeff
    Outputs
        mean of rmse, mean of mape, mean of mae
    """
    # Predict using single exponential smoothing, and compute error metrics also
    rmse = [] # root mean square error
    mape = [] # mean absolute percentage error
    mae = []  # mean absolute error
    preds_dict = {}
    
    for i in range(train_size, len(series)-H+1, int(H/2)):
        preds_list, seasonals = triple_exponential_smoothing(series[i-train_size:i], L, H, alpha, beta, gamma)
        
        rmse.append(get_rmse(series[i:i+H], preds_list))
        mape.append(get_mape(series[i:i+H], preds_list))
        mae.append(get_mae(series[i:i+H], preds_list))
        preds_dict[i] = preds_list
    
    return np.mean(rmse), np.mean(mape), np.mean(mae), preds_dict, alpha, beta, gamma   

def hyperparam_tune_alpha_beta_gamma_parallelized(series, train_size, L, H):
    """
    This is a parallelized implementation of hyperparam_tune_alpha_beta_gamma_finetune.
    Given a series, tune hyperparameter alpha, fit and predict
    Inputs
        series     : series to forecast, with length = (train_size + val_size)
        train_size : length of series to use as train ie. train set is series[:train_size]
        L          : period
        H          : forecast horizon
    Outputs
        optimum hyperparameters, error metrics dataframe
    """
    num_cores = multiprocessing.cpu_count()
    inputs = []
    alpha = alphaMin
    while alpha <= alphaMax:
        beta = betaMin
        while beta <= betaMax:
            gamma = gammaMin
            while gamma <= gammaMax:
                inputs.append((alpha, beta, gamma)) 
                gamma = gamma + gammaStep
            beta = beta + betaStep
        alpha = alpha + alphaStep
        
    results = Parallel(n_jobs=num_cores)(delayed(get_error_metrics2)(series, train_size, L, H, item[0], item[1], item[2]) for item in inputs)
    # results has format [(rmse_mean1, mape_mean1, mae_mean1, preds_dict1, alpha1, beta1, gamma1), (rmse_mean2, mape_mean2, mae_mean2, preds_dict2, alpha2, beta2, gamma2), ...]   
    
    err_dict = defaultdict(list)
    for item in results:
        # Append error metrics
        err_dict['rmse'].append(item[0])
        err_dict['mape'].append(item[1])
        err_dict['mae'].append(item[2])
        
        # Append alpha and beta
        err_dict['alpha'].append(item[4])
        err_dict['beta'].append(item[5])
        err_dict['gamma'].append(item[6])
         
    # Convert to dataframe
    err_df = pd.DataFrame(err_dict)
    
    # Get min RMSE
    rmse_min = err_df['rmse'].min()
    
    alpha_opt = err_df[err_df['rmse'] == rmse_min]['alpha'].values[0]
    beta_opt = err_df[err_df['rmse'] == rmse_min]['beta'].values[0]
    gamma_opt = err_df[err_df['rmse'] == rmse_min]['gamma'].values[0]
    print("alpha_opt = " + str(alpha_opt) + 
          ", beta_opt = " + str(beta_opt) + 
          ", gamma_opt = " + str(gamma_opt) + 
          ", rmse_min = " + str(rmse_min))
    
    return alpha_opt, beta_opt, gamma_opt, err_df

In [ ]:
# Convert Date column to datetime
df.loc[:, 'Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')

# Change all column headings to be lower case, and remove spacing
df.columns = [str(x).lower().replace(' ', '_') for x in df.columns]

# Sort by datetime
df.sort_values(by='date', inplace=True, ascending=True)

df.head(10)

In [ ]:
df['date'].min(), df['date'].max() 

In [ ]:
# Plot adjusted close over time
rcParams['figure.figsize'] = 10, 8 # width 10, height 8

ax = df.plot(x='date', y='adj_close', style='b-', grid=True)
ax.set_xlabel("date")
ax.set_ylabel("USD")

In [ ]:
i = 5000
# Predict
preds_list, seasonals = triple_exponential_smoothing(df['adj_close'][i-train_val_size:i].values, L, H, 0.3, 0.55, 0.6, True)
print("For forecast horizon %d, predicting on day %d, date %s, the RMSE is %f" % (H, i, df['date'][i], get_rmse(df[i:i+H]['adj_close'], preds_list[train_val_size:train_val_size+H])))
print("For forecast horizon %d, predicting on day %d, date %s, the mean MAPE is %f" % (H, i, df['date'][i], get_mape(df[i:i+H]['adj_close'], preds_list[train_val_size:train_val_size+H])))
print("For forecast horizon %d, predicting on day %d, date %s, the mean MAE is %f" % (H, i, df['date'][i], get_mae(df[i:i+H]['adj_close'], preds_list[train_val_size:train_val_size+H])))

In [ ]:
# Plot the predictions
rcParams['figure.figsize'] = 15, 11 # width 10, height 8
matplotlib.rcParams.update({'font.size': 14})
ax = df.plot(x='date', y='adj_close', color='black', grid=True)

# Plot the predictions
ax.plot(df['date'][i-train_val_size:i+H], preds_list, color='r')
ax.plot(df['date'][i:i+H], preds_list[train_val_size:train_val_size+H], color='blue')
    
ax.set_xlabel("date")
ax.set_ylabel("USD")
ax.legend(['adj_close', 'predictions'])
ax.set_ylim([min(min(preds_list[1:]), df[(df['date']>=df['date'][i]-timedelta(days=daysBackward)) & (df['date']<=df['date'][i]+timedelta(days=daysForward))]['adj_close'].min()), 
             max(max(preds_list[1:]), df[(df['date']>=df['date'][i]-timedelta(days=daysBackward)) & (df['date']<=df['date'][i]+timedelta(days=daysForward))]['adj_close'].max())])
ax.set_xlim([df['date'][i-train_val_size+1], df['date'][i]+timedelta(days=H)])

In [ ]:
# Plot the seasonals
rcParams['figure.figsize'] = 10, 8 # width 10, height 8
matplotlib.rcParams.update({'font.size': 14})

plt.plot(seasonals)
plt.grid()

In [ ]:
# Plot the seasonals with actual values on dual axes
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('date')
ax1.set_ylabel('USD', color=color)
ax1.plot(df['date'][:len(seasonals)], df['adj_close'][:len(seasonals)], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('seasonal components', color=color)  # we already handled the x-label with ax1
ax2.plot(df['date'][:len(seasonals)], seasonals, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.grid()
plt.show()